格子上の最短ベクトルの数え上げを行うアルゴリズム<br>
入力： $n$ 次元格子 $L$ の基底のGSO係数 $\mu_{i,j} (1 \leq j < i \leq n)$, 
GSOベクトルの二乗ノルム $B_i = \| \mathbb{b_i^\star} \|^2 (1 \leq i \leq n)$, 
数え上げ上界列 $R_1^2 \leq \cdots \leq R_n^2$<br>
出力：すべての $1 \leq k \leq n$ に対して $\| \pi_k(\mathbb{v}) \|^2 \leq R_{n+1-k}^2$ を満たす格子ベクトル<br>
　　　$\mathbb{v} = \sum_{i=1}^n v_i \mathbb{b_i} \in L$ の係数ベクトル $(v_1, \dots, v_n) \in \mathbb{Z}^n$ ( $v$ が存在する場合)

事前準備

In [49]:
import numpy as np
import math

def input_basis_form():
  print("Enter the dimension of the lattice.")
  n = int(input())
  print("---")
  print("Enter each basis vector of the lattice")
  B = np.array([list(map(int,input().split())) for _ in range(n)])
  return B

# 基底行列を入力するとそれらのGSOベクトルからなる行列とGSO係数からなる行列を出力するアルゴリズム
# サブルーチン用
def GSO(B):
  # Bの行数（基底ベクトルの本数）をnとする
  n = len(B)
  # Bと同じサイズのゼロ行列(浮動小数点数成分)を生成
  # ここにGSOの結果を入れていく予定
  B_star = np.zeros_like(B, dtype=float)
  # GSO係数行列のベースを作成
  mu = np.eye(n, dtype=float)

  for i in range(n):
    B_star[i] = B[i]
    for j in range(i):
      mu[i,j] = np.dot(B[i], B_star[j]) / np.dot(B_star[j], B_star[j])
      B_star[i] -= mu[i,j] * B_star[j]
  # B_star(GSOベクトル行列)とmu(GSO係数行列)を出力
  return B_star, mu

# 基底行列を入力するとそれらのGSOベクトルからなる行列とGSO係数からなる行列を出力するアルゴリズム
# メイン処理用
def GSO_main():
  B = input_basis_form()
  return GSO(B)

# GSO係数とGSOベクトルの二乗ノルムを出力するアルゴリズム
def GSO_coeffs_sqnorms(B):
  B_star,mu = GSO(B)
  B_sqnorm = np.array([np.dot(B_star[i],B_star[i]) for i in range(len(B))])
  return B_sqnorm, mu

# ランダムな整数係数フルランク行列を生成するアルゴリズム（やっつけバージョン）
def create_randint_fullrank_matrix():
  print("Enter the size of matrix")
  s = list(map(int, input().split()))
  row = s[0]
  col = s[1]
  while True:
    rand = np.random.randint(-100, 100, size=(row, col))
    if np.linalg.matrix_rank(rand) == row:
      return rand
    else:
      continue

数え上げ上界列を受け取るアルゴリズムを定義

In [50]:
def input_enum_upperbound_seq():
  print("---")
  print("Enter a parameter epsilon to define the enumeration upper bounds as:")
  print("R_k^2 = (k / n) * epsilon * ||b_1*||^2")
  epsilon = float(input())
  R_5 = B_sqnorm[0] * epsilon
  R = [(k+1)/5 * R_5 for k in range(5)]
  return R

本題のアルゴリズムを定義（サブルーチン用）

In [51]:
def enum_shortest_vec(B_sqnorm, mu, R):
  n = len(B_sqnorm)
  sigma = np.zeros(shape=(n+1,n), dtype=float)
  r = [i for i in range(n+1)]
  rho = [float(0) for i in range(n+1)]
  v = [0 for i in range(n)]
  c = [0 for i in range(n)]
  w = [0 for i in range(n)]
  v[0] = 1
  last_nonzero = 1
  k=0
  while True:
    rho[k] = rho[k+1] + (v[k] - c[k])**2 * B_sqnorm[k]
    if rho[k] <= R[n-1-k]:
      if k == 0:
        return v
      k -= 1
      r[k] = max(r[k+1], r[k])
      for i in range(r[k+1],k,-1):
        sigma[i,k] = sigma[i+1,k] + mu[i,k] * v[i]
      c[k] = -sigma[k+1,k]
      v[k] = round(c[k])
      w[k] = 1
    else:
      k += 1
      if k == n:
        return "none"
      r[k] = k
      if k >= last_nonzero:
        last_nonzero = k
        v[k] += 1
      else:
        if v[k] > c[k]:
          v[k] -= w[k]
        else:
          v[k] += w[k]
        w[k] += 1

実際に用いてみる

In [ ]:
B = np.array([[63, -14, -1, 84, 61],
              [74, -20, 23, -32, -52],
              [93, -46, -19, 0, -63],
              [93, 11, 13, 60, 52],
              [33, -93, 12, 57, -2]])
B_sqnorm, mu = GSO_coeffs_sqnorms(B)
print("The squared norms of the GSO vectors are")
print(B_sqnorm)
print("---")
print("The GSO coefficient matrix is")
print(mu)

R_5 = B_sqnorm[0] * 0.001

R = [(k+1)/5 * R_5 for k in range(5)]
print("---")
print("The upper bound sequence is")
print(R)
print("---")
#vol = math.sqrt(np.prod(B_sqnorm))
#upper_bound = math.sqrt(5) ** math.pow(vol, 1/5)
#R = [upper_bound for i in range(5)]
v = enum_shortest_vec(B_sqnorm, mu, R)
if v != "none":
  print("The coefficent vector is")
  print(v)
  print("---")
  print("The following vector was found.")
  print(np.dot(v, B))
else:
  print("No lattice vector was found.")

The squared norms of the GSO vectors are
[1.49430000e+04 1.00737428e+04 3.01527354e+03 7.03392732e+02
 3.41094435e-10]
---
The GSO coefficient matrix is
[[ 1.          0.          0.          0.          0.        ]
 [-0.06297263  1.          0.          0.          0.        ]
 [ 0.17928127  1.07305735  1.          0.          0.        ]
 [ 0.93046912  0.31890545 -0.43777128  1.          0.        ]
 [ 0.53770996  0.33393597  0.72786965 -2.94334071  1.        ]]
---
The upper bound sequence is
[np.float64(0.29886), np.float64(0.59772), np.float64(0.8965799999999999), np.float64(1.19544), np.float64(1.4943)]
---
No lattice vector was found.


標準入力対応版

In [ ]:
B = input_basis_form()
R = input_enum_upperbound_seq()
B_sqnorm, mu = GSO_coeffs_sqnorms(B)
print("The squared norms of the GSO vectors are")
print(B_sqnorm)
print("---")
print("The GSO coefficient matrix is")
print(mu)
print("---")
print("The upper bound sequence is")
print(R)
print("---")
v = enum_shortest_vec(B_sqnorm, mu, R)
if v != "none":
  print("The coefficent vector is")
  print(v)
  print("---")
  print("The following vector was found.")
  print(np.dot(v, B))
else:
  print("No lattice vector was found.")

Enter the dimension of the lattice.
---
Enter each basis vector of the lattice
Enter a parameter epsilon to define the enumeration upper bounds as:
R_k^2 = (k / n) * epsilon * ||b_1*||^2
The squared norms of the GSO vectors are
[8510.         5155.32655699 3213.1403949  2133.17518476]
---
The GSO coefficient matrix is
[[1.         0.         0.         0.        ]
 [0.5159812  1.         0.         0.        ]
 [1.09353702 0.56063936 1.         0.        ]
 [0.55945946 0.97344241 0.50858349 1.        ]]
---
The upper bound sequence is
[np.float64(851.0), np.float64(1702.0), np.float64(2553.0), np.float64(3404.0), np.float64(4255.0)]
---
The coefficent vector is
none
---
No lattice vector was found.


ランダムなフルランク行列（基底行列）を生成し、その基底で張られる格子上の短いベクトルを見つけるアルゴリズム

In [52]:
B = create_randint_fullrank_matrix()
print("The basis matrix is")
print(B)
print("---")
R = input_enum_upperbound_seq()
B_sqnorm, mu = GSO_coeffs_sqnorms(B)
print("The squared norms of the GSO vectors are")
print(B_sqnorm)
print("---")
print("The GSO coefficient matrix is")
print(mu)
print("---")
print("The upper bound sequence is")
print(R)
print("---")
v = enum_shortest_vec(B_sqnorm, mu, R)
if v != "none":
  print("The coefficent vector is")
  print(v)
  print("---")
  print("The following vector was found.")
  print(np.dot(v, B))
else:
  print("No lattice vector was found.")

Enter the size of matrix
The basis matrix is
[[ 49 -55  36  57  -4]
 [ 44  55 -56  79  71]
 [ 98  93  74  24  23]
 [  5  47 -14  64  -6]
 [-88 -77   1 -47 -65]]
---
---
Enter a parameter epsilon to define the enumeration upper bounds as:
R_k^2 = (k / n) * epsilon * ||b_1*||^2
The squared norms of the GSO vectors are
[ 9987.         19200.81275658 19905.06670737  2969.95787367
    42.17795973]
---
The GSO coefficient matrix is
[[ 1.          0.          0.          0.          0.        ]
 [ 0.13357365  1.          0.          0.          0.        ]
 [ 0.36317212  0.43370708  1.          0.          0.        ]
 [ 0.08290778  0.42229468  0.07061632  1.          0.        ]
 [-0.24632022 -0.84175649 -0.52402967  0.36190422  1.        ]]
---
The upper bound sequence is
[np.float64(597.7200000000001), np.float64(1195.4400000000003), np.float64(1793.16), np.float64(2390.8800000000006), np.float64(2988.6000000000004)]
---
No lattice vector was found.


見つからないと悲しいので、見つからなくなるギリギリまでRを減らすアルゴリズムにしてみる

In [61]:
B = create_randint_fullrank_matrix()
print("The basis matrix is")
print(B)
print("---")
B_sqnorm, mu = GSO_coeffs_sqnorms(B)
print("The squared norms of the GSO vectors are")
print(B_sqnorm)
print("---")
print("The GSO coefficient matrix is")
print(mu)
print("---")

epsilon = 1
min_v= [0 for i in range(len(B))]
min_v[0] = 1
while True:
  R_5 = B_sqnorm[0] * epsilon
  R = [(k+1)/5 * R_5 for k in range(5)]
  v = enum_shortest_vec(B_sqnorm, mu, R)
  if v != "none":
    min_v = v
    epsilon -= 0.001
    continue
  else:
    break
print("We use the following upper bound sequence as:")
print("R_k^2 = (k / n) * epsilon * ||b_1*||^2")
print("almost minimal epsilon is")
print(epsilon)
print("---")
print("The upper bound sequence is")
print(R)
print("---")
print("The coefficent vector is")
print(min_v)
print("---")
print("The following vector was found.")
print(np.dot(min_v, B))

Enter the size of matrix
The basis matrix is
[[ 74  60  97 -83 -95]
 [ 31 -81  96 -87  57]
 [-51  76 -52  98  36]
 [ 92  12  83 -65 -77]
 [-21  38 -27 -77 -90]]
---
The squared norms of the GSO vectors are
[34399.         25429.87121719  5541.19165311  1157.64354027
   160.38654472]
---
The GSO coefficient matrix is
[[ 1.          0.          0.          0.          0.        ]
 [ 0.24861188  1.          0.          0.          0.        ]
 [-0.45966453 -0.60055157  1.          0.          0.        ]
 [ 0.82237856  0.16048129 -0.32189613  1.          0.        ]
 [ 0.37931335 -0.31442895 -0.76287565 -2.23724937  1.        ]]
---
We use the following upper bound sequence as:
R_k^2 = (k / n) * epsilon * ||b_1*||^2
almost minimal epsilon is
0.06699999999999917
---
The upper bound sequence is
[np.float64(460.9465999999943), np.float64(921.8931999999886), np.float64(1382.839799999983), np.float64(1843.7863999999772), np.float64(2304.7329999999715)]
---
The coefficent vector is
[-2, 1, 2, 3